# Telecom Churn Case Study

In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

For many incumbent operators, retaining high profitable customers is the number one business
goal. To reduce customer churn, telecom companies need to predict which customers are at high risk of churn. In this project, you will analyze customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn.

In this competition, your goal is to build a machine learning model that is able to predict churning customers based on the features provided for their usage.

---

If you are reading this notebook from Github, there are some features that may not be working properly, to have the notebook works as expected, you can read this notebook through jupyter notebook viewer on the following link:
<a href="https://nbviewer.org/github/hendrauzia/ljmu-tccs/blob/master/code/Telecom%20Churn%20Case%20Study.ipynb">Telecom Churn Case Study</a>